In [1]:
import torch

from src.utils_data import load_PeMS04_flow_data, preprocess_PeMS_data, createLoaders
from src.models import TGCN
from src.utils_training import train_model, testmodel

import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
def mask_unknown_nodes(df_PeMS, list_nodes_known=None):
    if list_nodes_known is None:
        return df_PeMS, None
    mask = []
    temp = df_PeMS.copy()
    nb_values = temp.shape[0]
    for node in temp:
        if(node not in list_nodes_known):
            temp[node] = [0 for _ in range(nb_values)]
            mask.append(0)
        else:
            mask.append(1)
    return temp, torch.tensor(mask).float()

In [3]:
def result_prediction(predictions, actuals):
    from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
    import numpy as np
    
    indices_by_month = []
    EPSILON = 1e-5
    # Créer une liste vide pour stocker les données du tableau
    data = []
    y_pred = predictions[:]
    y_true = actuals[:]

    signe = "-" if np.mean(y_pred - y_true) < 0 else "+"
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = mean_absolute_percentage_error(y_true, y_pred)*100
    if (mape > 1 or mape < 0):
        mape = "ERROR"
    smape = np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))*100
    maape =  np.mean(np.arctan(np.abs((y_true - y_pred) / (y_true + EPSILON))))*100
    
    return [signe, mae, rmse, mape, smape, maape]

In [4]:
# import numpy as np
# from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error


# _window_size = 7
# horizon = 1
# _stride = 1

# # TGCN
# num_epochs_TGCN = 400

# n_neighbors = 10

# df_PeMS_old, df_distance  = load_PeMS04_flow_data()
# df_PeMS, adjacency_matrix_PeMS, meanstd_dict = preprocess_PeMS_data(df_PeMS_old, df_distance, init_node=0, n_neighbors=n_neighbors, center_and_reduce=True)
# print(df_PeMS.columns)
# node_0, mask_node_0 = mask_unknown_nodes(df_PeMS=df_PeMS, list_nodes_known=[68, 62, 242, 243, 117, 28, 170])
# node_1, mask_node_1 = mask_unknown_nodes(df_PeMS=df_PeMS, list_nodes_known=[0, 173, 92, 57])

# # TGCN Model
# model_TGCN = TGCN(adjacency_matrix_PeMS, hidden_dim=32, output_size=len(df_PeMS.columns))
# train_loader_TGCN, val_loader_TGCN, test_loader_TGCN, _ = createLoaders(node_0, window_size=_window_size, stride=_stride, prediction_horizon=horizon)
# model_path = "./test1.pkl"
# best_model , train_losses, valid_losses = train_model(model_TGCN, train_loader_TGCN, val_loader_TGCN, model_path=model_path, mask=mask_node_0, num_epochs=num_epochs_TGCN, remove=False)

# y_true, y_pred = testmodel(best_model=best_model, test_loader=test_loader_TGCN, meanstd_dict=meanstd_dict)
# plt.figure(figsize=(49,9))
# plt.plot(y_true[:,:,1])
# plt.plot(y_pred[:,:,1])
# plt.show()
# print(y_true.shape)
# rmse = 0
# for i in range(n_neighbors+1):
#     rmse = rmse + np.sqrt(mean_squared_error(y_true[:,:,i], y_pred[:,:,i]))
# print(rmse/n_neighbors+1)

# # TGCN Model
# model_TGCN = TGCN(adjacency_matrix_PeMS, hidden_dim=32, output_size=len(df_PeMS.columns))
# train_loader_TGCN, val_loader_TGCN, test_loader_TGCN, _ = createLoaders(node_0, window_size=_window_size, stride=_stride, prediction_horizon=horizon)
# model_path = "./test2.pkl"
# best_model , train_losses, valid_losses = train_model(model_TGCN, train_loader_TGCN, val_loader_TGCN, model_path=model_path, mask=None, num_epochs=num_epochs_TGCN, remove=False)

# y_true, y_pred = testmodel(best_model=best_model, test_loader=test_loader_TGCN, meanstd_dict=meanstd_dict)
# plt.figure(figsize=(49,9))
# plt.plot(y_true[:,:,1])
# plt.plot(y_pred[:,:,1])
# plt.show()
# rmse = 0
# for i in range(n_neighbors+1):
#     rmse = rmse + np.sqrt(mean_squared_error(y_true[:,:,i], y_pred[:,:,i]))
# print(rmse/n_neighbors+1)

## (All variables) vs (Dummy Variables)

In [5]:
import numpy as np
from sklearn.metrics import mean_squared_error

_window_size = 7
horizon = 1
_stride = 1

# TGCN
num_epochs_TGCN = 350

n_neighbors = 2
df_PeMS_old, df_distance  = load_PeMS04_flow_data()
df_PeMS, adjacency_matrix_PeMS, meanstd_dict = preprocess_PeMS_data(df_PeMS_old, df_distance, init_node=0, n_neighbors=n_neighbors, center_and_reduce=True)
print(df_PeMS.columns)
node_0, _ = mask_unknown_nodes(df_PeMS=df_PeMS, list_nodes_known=[117, 0, 92])
node_1, mask_node_1 = mask_unknown_nodes(df_PeMS=df_PeMS, list_nodes_known=[117, 0])

# TGCN Model
model_TGCN = TGCN(adjacency_matrix_PeMS, hidden_dim=32, output_size=len(df_PeMS.columns))
train_loader_TGCN, val_loader_TGCN, test_loader_TGCN, _ = createLoaders(node_0, window_size=_window_size, stride=_stride, prediction_horizon=horizon)
model_path = "./test3_3.pkl"
best_model , train_losses, valid_losses = train_model(model_TGCN, train_loader_TGCN, val_loader_TGCN, model_path=model_path, mask=None, num_epochs=num_epochs_TGCN, remove=False)
y_true, y_pred = testmodel(best_model=best_model, test_loader=test_loader_TGCN, meanstd_dict=meanstd_dict, sensor_order_list=node_0.columns)
rmse = 0
for i in range(n_neighbors+1):
    rmse = rmse + np.sqrt(mean_squared_error(y_true[:,:,i], y_pred[:,:,i]))
print(rmse/n_neighbors+1)

model_TGCN = TGCN(adjacency_matrix_PeMS, hidden_dim=32, output_size=len(df_PeMS.columns))
train_loader_TGCN, val_loader_TGCN, test_loader_TGCN, _ = createLoaders(node_1, window_size=_window_size, stride=_stride, prediction_horizon=horizon)
model_path = "./test2_3.pkl"
best_model , train_losses, valid_losses = train_model(model_TGCN, train_loader_TGCN, val_loader_TGCN, model_path=model_path, mask=None, num_epochs=num_epochs_TGCN, remove=False)
y_true, y_pred = testmodel(best_model=best_model, test_loader=test_loader_TGCN, meanstd_dict=meanstd_dict, sensor_order_list=node_1.columns)
rmse = 0
for i in range(n_neighbors+1):
    rmse = rmse + np.sqrt(mean_squared_error(y_true[:,:,i], y_pred[:,:,i]))
print(rmse/n_neighbors+1)

c:\Users\gcogoni\Documents\Stage\Federated-Traffic-Flow-Forecasting\src\utils_graph.py:98: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  return nx.adjacency_matrix(graph, nodelist=nodes_order, weight=None).toarray()


Int64Index([117, 0, 92], dtype='int64')
Epoch 1/350, Training Loss: 0.1123, Validation Loss: 0.0330
Epoch 2/350, Training Loss: 0.0507, Validation Loss: 0.0190
Epoch 3/350, Training Loss: 0.0372, Validation Loss: 0.0150
Epoch 4/350, Training Loss: 0.0329, Validation Loss: 0.0138
Epoch 5/350, Training Loss: 0.0308, Validation Loss: 0.0131
Epoch 6/350, Training Loss: 0.0293, Validation Loss: 0.0128
Epoch 7/350, Training Loss: 0.0280, Validation Loss: 0.0129
Epoch 8/350, Training Loss: 0.0266, Validation Loss: 0.0132
Epoch 9/350, Training Loss: 0.0254, Validation Loss: 0.0137
Epoch 10/350, Training Loss: 0.0243, Validation Loss: 0.0139
Epoch 11/350, Training Loss: 0.0232, Validation Loss: 0.0137
Epoch 12/350, Training Loss: 0.0222, Validation Loss: 0.0132
Epoch 13/350, Training Loss: 0.0211, Validation Loss: 0.0128
Epoch 14/350, Training Loss: 0.0200, Validation Loss: 0.0124
Epoch 15/350, Training Loss: 0.0189, Validation Loss: 0.0123
Epoch 16/350, Training Loss: 0.0178, Validation Loss: 

In [6]:
import numpy as np
from sklearn.metrics import mean_squared_error


_window_size = 7
horizon = 1
_stride = 1

# TGCN
num_epochs_TGCN = 350

n_neighbors = 5
df_PeMS_old, df_distance  = load_PeMS04_flow_data()
df_PeMS, adjacency_matrix_PeMS, meanstd_dict = preprocess_PeMS_data(df_PeMS_old, df_distance, init_node=0, n_neighbors=n_neighbors, center_and_reduce=True)
print(df_PeMS.columns)
node_0, _ = mask_unknown_nodes(df_PeMS=df_PeMS, list_nodes_known=[243, 117, 28, 0, 173, 92])
node_1, mask_node_1 = mask_unknown_nodes(df_PeMS=df_PeMS, list_nodes_known=[243, 117, 0])

# TGCN Model
model_TGCN = TGCN(adjacency_matrix_PeMS, hidden_dim=32, output_size=len(df_PeMS.columns))
train_loader_TGCN, val_loader_TGCN, test_loader_TGCN, _ = createLoaders(node_0, window_size=_window_size, stride=_stride, prediction_horizon=horizon)
model_path = "./test5_5.pkl"
best_model , train_losses, valid_losses = train_model(model_TGCN, train_loader_TGCN, val_loader_TGCN, model_path=model_path, mask=None, num_epochs=num_epochs_TGCN, remove=False)
y_true, y_pred = testmodel(best_model=best_model, test_loader=test_loader_TGCN, meanstd_dict=meanstd_dict, sensor_order_list=node_0.columns)
rmse = 0
for i in range(n_neighbors+1):
    rmse = rmse + np.sqrt(mean_squared_error(y_true[:,:,i], y_pred[:,:,i]))
print(rmse/n_neighbors+1)

model_TGCN = TGCN(adjacency_matrix_PeMS, hidden_dim=32, output_size=len(df_PeMS.columns))
train_loader_TGCN, val_loader_TGCN, test_loader_TGCN, _ = createLoaders(node_1, window_size=_window_size, stride=_stride, prediction_horizon=horizon)
model_path = "./test3_5.pkl"
best_model , train_losses, valid_losses = train_model(model_TGCN, train_loader_TGCN, val_loader_TGCN, model_path=model_path, mask=None, num_epochs=num_epochs_TGCN, remove=False)
y_true, y_pred = testmodel(best_model=best_model, test_loader=test_loader_TGCN, meanstd_dict=meanstd_dict, sensor_order_list=node_1.columns)
rmse = 0
for i in range(n_neighbors+1):
    rmse = rmse + np.sqrt(mean_squared_error(y_true[:,:,i], y_pred[:,:,i]))
print(rmse/n_neighbors+1)

c:\Users\gcogoni\Documents\Stage\Federated-Traffic-Flow-Forecasting\src\utils_graph.py:98: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  return nx.adjacency_matrix(graph, nodelist=nodes_order, weight=None).toarray()


Int64Index([243, 117, 28, 0, 173, 92], dtype='int64')
Epoch 1/350, Training Loss: 0.2216, Validation Loss: 0.0742
Epoch 2/350, Training Loss: 0.0879, Validation Loss: 0.0598
Epoch 3/350, Training Loss: 0.0718, Validation Loss: 0.0481
Epoch 4/350, Training Loss: 0.0633, Validation Loss: 0.0452
Epoch 5/350, Training Loss: 0.0598, Validation Loss: 0.0434
Epoch 6/350, Training Loss: 0.0564, Validation Loss: 0.0411
Epoch 7/350, Training Loss: 0.0516, Validation Loss: 0.0378
Epoch 8/350, Training Loss: 0.0448, Validation Loss: 0.0337
Epoch 9/350, Training Loss: 0.0378, Validation Loss: 0.0305
Epoch 10/350, Training Loss: 0.0331, Validation Loss: 0.0289
Epoch 11/350, Training Loss: 0.0302, Validation Loss: 0.0264
Epoch 12/350, Training Loss: 0.0285, Validation Loss: 0.0243
Epoch 13/350, Training Loss: 0.0268, Validation Loss: 0.0237
Epoch 14/350, Training Loss: 0.0255, Validation Loss: 0.0244
Epoch 15/350, Training Loss: 0.0237, Validation Loss: 0.0216
Epoch 16/350, Training Loss: 0.0229, Val

In [7]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error


_window_size = 7
horizon = 1
_stride = 1

# TGCN
num_epochs_TGCN = 350

n_neighbors = 12
df_PeMS_old, df_distance  = load_PeMS04_flow_data()
df_PeMS, adjacency_matrix_PeMS, meanstd_dict = preprocess_PeMS_data(df_PeMS_old, df_distance, init_node=0, n_neighbors=n_neighbors, center_and_reduce=True)
print(df_PeMS.columns)
node_0, _ = mask_unknown_nodes(df_PeMS=df_PeMS, list_nodes_known=[68, 62, 242, 243, 117, 28, 170, 0, 173, 271, 92, 57, 97])
node_1, mask_node_1 = mask_unknown_nodes(df_PeMS=df_PeMS, list_nodes_known=[68, 62, 242, 243, 117, 28, 170])

# TGCN Model
model_TGCN = TGCN(adjacency_matrix_PeMS, hidden_dim=32, output_size=len(df_PeMS.columns))
train_loader_TGCN, val_loader_TGCN, test_loader_TGCN, _ = createLoaders(node_0, window_size=_window_size, stride=_stride, prediction_horizon=horizon)
model_path = "./test13_13.pkl"

best_model , train_losses, valid_losses = train_model(model_TGCN, train_loader_TGCN, val_loader_TGCN, model_path=model_path, mask=None, num_epochs=num_epochs_TGCN, remove=False)
y_true, y_pred = testmodel(best_model=best_model, test_loader=test_loader_TGCN, meanstd_dict=meanstd_dict, sensor_order_list=node_0.columns)
rmse = 0
for i in range(n_neighbors+1):
    rmse = rmse + np.sqrt(mean_squared_error(y_true[:,:,i], y_pred[:,:,i]))
print(rmse/n_neighbors+1)

model_TGCN = TGCN(adjacency_matrix_PeMS, hidden_dim=32, output_size=len(df_PeMS.columns))
train_loader_TGCN, val_loader_TGCN, test_loader_TGCN, _ = createLoaders(node_1, window_size=_window_size, stride=_stride, prediction_horizon=horizon)
model_path = "./test7_13.pkl"
best_model , train_losses, valid_losses = train_model(model_TGCN, train_loader_TGCN, val_loader_TGCN, model_path=model_path, mask=None, num_epochs=num_epochs_TGCN, remove=False)
y_true, y_pred = testmodel(best_model=best_model, test_loader=test_loader_TGCN, meanstd_dict=meanstd_dict, sensor_order_list=node_1.columns)
rmse = 0
for i in range(n_neighbors+1):
    rmse = rmse + np.sqrt(mean_squared_error(y_true[:,:,i], y_pred[:,:,i]))
print(rmse/n_neighbors+1)

c:\Users\gcogoni\Documents\Stage\Federated-Traffic-Flow-Forecasting\src\utils_graph.py:98: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  return nx.adjacency_matrix(graph, nodelist=nodes_order, weight=None).toarray()


Int64Index([68, 62, 242, 243, 117, 28, 170, 0, 173, 271, 92, 57, 97], dtype='int64')
Epoch 1/350, Training Loss: 0.1854, Validation Loss: 0.0955
Epoch 2/350, Training Loss: 0.1014, Validation Loss: 0.0828
Epoch 3/350, Training Loss: 0.0874, Validation Loss: 0.0708
Epoch 4/350, Training Loss: 0.0790, Validation Loss: 0.0662
Epoch 5/350, Training Loss: 0.0746, Validation Loss: 0.0625
Epoch 6/350, Training Loss: 0.0703, Validation Loss: 0.0581
Epoch 7/350, Training Loss: 0.0629, Validation Loss: 0.0507
Epoch 8/350, Training Loss: 0.0530, Validation Loss: 0.0428
Epoch 9/350, Training Loss: 0.0451, Validation Loss: 0.0382
Epoch 10/350, Training Loss: 0.0413, Validation Loss: 0.0360
Epoch 11/350, Training Loss: 0.0391, Validation Loss: 0.0338
Epoch 12/350, Training Loss: 0.0365, Validation Loss: 0.0348
Epoch 13/350, Training Loss: 0.0348, Validation Loss: 0.0316
Epoch 14/350, Training Loss: 0.0334, Validation Loss: 0.0300
Epoch 15/350, Training Loss: 0.0323, Validation Loss: 0.0290
Epoch 16/

## (All variables) vs (MASK + Dummy Variables)

In [8]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error


_window_size = 7
horizon = 1
_stride = 1

# TGCN
num_epochs_TGCN = 400

n_neighbors = 2
df_PeMS_old, df_distance  = load_PeMS04_flow_data()
df_PeMS, adjacency_matrix_PeMS, meanstd_dict = preprocess_PeMS_data(df_PeMS_old, df_distance, init_node=0, n_neighbors=n_neighbors, center_and_reduce=True)
print(df_PeMS.columns)
node_0, _ = mask_unknown_nodes(df_PeMS=df_PeMS, list_nodes_known=[117, 0, 92])
node_1, mask_node_1 = mask_unknown_nodes(df_PeMS=df_PeMS, list_nodes_known=[117, 0])

# TGCN Model
model_path = "./test3_3.pkl"
best_model = TGCN(adjacency_matrix_PeMS, hidden_dim=32, output_size=len(df_PeMS.columns)).load_state_dict(torch.load(model_path))
y_true, y_pred = testmodel(best_model=best_model, test_loader=test_loader_TGCN, meanstd_dict=meanstd_dict, sensor_order_list=node_0.columns)
rmse = 0
for i in range(n_neighbors+1):
    rmse = rmse + np.sqrt(mean_squared_error(y_true[:,:,i], y_pred[:,:,i]))
print(rmse/n_neighbors+1)

model_TGCN = TGCN(adjacency_matrix_PeMS, hidden_dim=32, output_size=len(df_PeMS.columns))
train_loader_TGCN, val_loader_TGCN, test_loader_TGCN, _ = createLoaders(node_1, window_size=_window_size, stride=_stride, prediction_horizon=horizon)
model_path = "./test2_3_mask.pkl"
best_model , train_losses, valid_losses = train_model(model_TGCN, train_loader_TGCN, val_loader_TGCN, model_path=model_path, mask=mask_node_1, num_epochs=num_epochs_TGCN, remove=False)
y_true, y_pred = testmodel(best_model=best_model, test_loader=test_loader_TGCN, meanstd_dict=meanstd_dict, sensor_order_list=node_1.columns)
rmse = 0
for i in range(n_neighbors+1):
    rmse = rmse + np.sqrt(mean_squared_error(y_true[:,:,i], y_pred[:,:,i]))
print(rmse/n_neighbors+1)

c:\Users\gcogoni\Documents\Stage\Federated-Traffic-Flow-Forecasting\src\utils_graph.py:98: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  return nx.adjacency_matrix(graph, nodelist=nodes_order, weight=None).toarray()


Int64Index([117, 0, 92], dtype='int64')


AttributeError: '_IncompatibleKeys' object has no attribute 'double'

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error


_window_size = 7
horizon = 1
_stride = 1

# TGCN
num_epochs_TGCN = 400

n_neighbors = 5
df_PeMS_old, df_distance  = load_PeMS04_flow_data()
df_PeMS, adjacency_matrix_PeMS, meanstd_dict = preprocess_PeMS_data(df_PeMS_old, df_distance, init_node=0, n_neighbors=n_neighbors, center_and_reduce=True)
print(df_PeMS.columns)
node_0, _ = mask_unknown_nodes(df_PeMS=df_PeMS, list_nodes_known=[243, 117, 28, 0, 173, 92])
node_1, mask_node_1 = mask_unknown_nodes(df_PeMS=df_PeMS, list_nodes_known=[243, 117, 0])

# TGCN Model
model_path = "./test5_5.pkl"
best_model = TGCN(adjacency_matrix_PeMS, hidden_dim=32, output_size=len(df_PeMS.columns)).load_state_dict(torch.load(model_path))
y_true, y_pred = testmodel(best_model=best_model, test_loader=test_loader_TGCN, meanstd_dict=meanstd_dict, sensor_order_list=node_0.columns)

rmse = 0
for i in range(n_neighbors+1):
    rmse = rmse + np.sqrt(mean_squared_error(y_true[:,:,i], y_pred[:,:,i]))
print(rmse/n_neighbors+1)

model_TGCN = TGCN(adjacency_matrix_PeMS, hidden_dim=32, output_size=len(df_PeMS.columns))
train_loader_TGCN, val_loader_TGCN, test_loader_TGCN, _ = createLoaders(node_1, window_size=_window_size, stride=_stride, prediction_horizon=horizon)
model_path = "./test2_3_mask.pkl"
best_model , train_losses, valid_losses = train_model(model_TGCN, train_loader_TGCN, val_loader_TGCN, model_path=model_path, mask=mask_node_1, num_epochs=num_epochs_TGCN, remove=False)
y_true, y_pred = testmodel(best_model=best_model, test_loader=test_loader_TGCN, meanstd_dict=meanstd_dict, sensor_order_list=node_1.columns)
rmse = 0
for i in range(n_neighbors+1):
    rmse = rmse + np.sqrt(mean_squared_error(y_true[:,:,i], y_pred[:,:,i]))
print(rmse/n_neighbors+1)

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error


_window_size = 7
horizon = 1
_stride = 1

# TGCN
num_epochs_TGCN = 400

n_neighbors = 12
df_PeMS_old, df_distance  = load_PeMS04_flow_data()
df_PeMS, adjacency_matrix_PeMS, meanstd_dict = preprocess_PeMS_data(df_PeMS_old, df_distance, init_node=0, n_neighbors=n_neighbors, center_and_reduce=True)
print(df_PeMS.columns)
node_0, _ = mask_unknown_nodes(df_PeMS=df_PeMS, list_nodes_known=[68, 62, 242, 243, 117, 28, 170, 0, 173, 271, 92, 57, 97])
node_1, mask_node_1 = mask_unknown_nodes(df_PeMS=df_PeMS, list_nodes_known=[68, 62, 242, 243, 117, 28, 170])

# TGCN Model
model_path = "./test13_13.pkl"
best_model = TGCN(adjacency_matrix_PeMS, hidden_dim=32, output_size=len(df_PeMS.columns)).load_state_dict(torch.load(model_path))
y_true, y_pred = testmodel(best_model=best_model, test_loader=test_loader_TGCN, meanstd_dict=meanstd_dict, sensor_order_list=node_0.columns)
rmse = 0
for i in range(n_neighbors+1):
    rmse = rmse + np.sqrt(mean_squared_error(y_true[:,:,i], y_pred[:,:,i]))
print(rmse/n_neighbors+1)

model_TGCN = TGCN(adjacency_matrix_PeMS, hidden_dim=32, output_size=len(df_PeMS.columns))
train_loader_TGCN, val_loader_TGCN, test_loader_TGCN, _ = createLoaders(node_1, window_size=_window_size, stride=_stride, prediction_horizon=horizon)
model_path = "./test7_13_mask.pkl"
best_model , train_losses, valid_losses = train_model(model_TGCN, train_loader_TGCN, val_loader_TGCN, model_path=model_path, mask=mask_node_1, num_epochs=num_epochs_TGCN, remove=False)
y_true, y_pred = testmodel(best_model=best_model, test_loader=test_loader_TGCN, meanstd_dict=meanstd_dict, sensor_order_list=node_1.columns)
rmse = 0
for i in range(n_neighbors+1):
    rmse = rmse + np.sqrt(mean_squared_error(y_true[:,:,i], y_pred[:,:,i]))
print(rmse/n_neighbors+1)

##  (Dummy Variables) VS (MASK + Dummy Variables)

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error


_window_size = 7
horizon = 1
_stride = 1

# TGCN
num_epochs_TGCN = 400

n_neighbors = 2
df_PeMS_old, df_distance  = load_PeMS04_flow_data()
df_PeMS, adjacency_matrix_PeMS, meanstd_dict = preprocess_PeMS_data(df_PeMS_old, df_distance, init_node=0, n_neighbors=n_neighbors, center_and_reduce=True)
print(df_PeMS.columns)
node_0, mask_node_0 = mask_unknown_nodes(df_PeMS=df_PeMS, list_nodes_known=[117, 0])
node_1, mask_node_1 = mask_unknown_nodes(df_PeMS=df_PeMS, list_nodes_known=[117, 0])

# TGCN Model
model_TGCN = TGCN(adjacency_matrix_PeMS, hidden_dim=32, output_size=len(df_PeMS.columns))
train_loader_TGCN, val_loader_TGCN, test_loader_TGCN, _ = createLoaders(node_0, window_size=_window_size, stride=_stride, prediction_horizon=horizon)
model_path = "./test2_2_without_mask.pkl"

best_model , train_losses, valid_losses = train_model(model_TGCN, train_loader_TGCN, val_loader_TGCN, model_path=model_path, mask=None, num_epochs=num_epochs_TGCN, remove=False)
y_true, y_pred = testmodel(best_model=best_model, test_loader=test_loader_TGCN, meanstd_dict=meanstd_dict, sensor_order_list=node_0.columns)
rmse = 0
for i in range(n_neighbors+1):
    rmse = rmse + np.sqrt(mean_squared_error(y_true[:,:,i], y_pred[:,:,i]))
print(rmse/n_neighbors+1)

model_TGCN = TGCN(adjacency_matrix_PeMS, hidden_dim=32, output_size=len(df_PeMS.columns))
train_loader_TGCN, val_loader_TGCN, test_loader_TGCN, _ = createLoaders(node_1, window_size=_window_size, stride=_stride, prediction_horizon=horizon)
model_path = "./test2_2_mask.pkl"
best_model , train_losses, valid_losses = train_model(model_TGCN, train_loader_TGCN, val_loader_TGCN, model_path=model_path, mask=mask_node_1, num_epochs=num_epochs_TGCN, remove=False)
y_true, y_pred = testmodel(best_model=best_model, test_loader=test_loader_TGCN, meanstd_dict=meanstd_dict, sensor_order_list=node_1.columns)
rmse = 0
for i in range(n_neighbors+1):
    rmse = rmse + np.sqrt(mean_squared_error(y_true[:,:,i], y_pred[:,:,i]))
print(rmse/n_neighbors+1)

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error


_window_size = 7
horizon = 1
_stride = 1

# TGCN
num_epochs_TGCN = 400

n_neighbors = 5
df_PeMS_old, df_distance  = load_PeMS04_flow_data()
df_PeMS, adjacency_matrix_PeMS, meanstd_dict = preprocess_PeMS_data(df_PeMS_old, df_distance, init_node=0, n_neighbors=n_neighbors, center_and_reduce=True)
print(df_PeMS.columns)
node_0, _ = mask_unknown_nodes(df_PeMS=df_PeMS, list_nodes_known=[243, 117, 28])
node_1, mask_node_1 = mask_unknown_nodes(df_PeMS=df_PeMS, list_nodes_known=[243, 117, 0])

# TGCN Model
model_TGCN = TGCN(adjacency_matrix_PeMS, hidden_dim=32, output_size=len(df_PeMS.columns))
train_loader_TGCN, val_loader_TGCN, test_loader_TGCN, _ = createLoaders(node_0, window_size=_window_size, stride=_stride, prediction_horizon=horizon)
model_path = "./test3_3_without_mask.pkl"

best_model , train_losses, valid_losses = train_model(model_TGCN, train_loader_TGCN, val_loader_TGCN, model_path=model_path, mask=None, num_epochs=num_epochs_TGCN, remove=False)
y_true, y_pred = testmodel(best_model=best_model, test_loader=test_loader_TGCN, meanstd_dict=meanstd_dict, sensor_order_list=node_0.columns)
rmse = 0
for i in range(n_neighbors+1):
    rmse = rmse + np.sqrt(mean_squared_error(y_true[:,:,i], y_pred[:,:,i]))
print(rmse/n_neighbors+1)

model_TGCN = TGCN(adjacency_matrix_PeMS, hidden_dim=32, output_size=len(df_PeMS.columns))
train_loader_TGCN, val_loader_TGCN, test_loader_TGCN, _ = createLoaders(node_1, window_size=_window_size, stride=_stride, prediction_horizon=horizon)
model_path = "./test3_3_mask.pkl"
best_model , train_losses, valid_losses = train_model(model_TGCN, train_loader_TGCN, val_loader_TGCN, model_path=model_path, mask=mask_node_1, num_epochs=num_epochs_TGCN, remove=False)
y_true, y_pred = testmodel(best_model=best_model, test_loader=test_loader_TGCN, meanstd_dict=meanstd_dict, sensor_order_list=node_1.columns)
rmse = 0
for i in range(n_neighbors+1):
    rmse = rmse + np.sqrt(mean_squared_error(y_true[:,:,i], y_pred[:,:,i]))
print(rmse/n_neighbors+1)

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error


_window_size = 7
horizon = 1
_stride = 1

# TGCN
num_epochs_TGCN = 400

n_neighbors = 12
df_PeMS_old, df_distance  = load_PeMS04_flow_data()
df_PeMS, adjacency_matrix_PeMS, meanstd_dict = preprocess_PeMS_data(df_PeMS_old, df_distance, init_node=0, n_neighbors=n_neighbors, center_and_reduce=True)
print(df_PeMS.columns)
node_0, _ = mask_unknown_nodes(df_PeMS=df_PeMS, list_nodes_known=[68, 62, 242, 243, 117, 28, 170])
node_1, mask_node_1 = mask_unknown_nodes(df_PeMS=df_PeMS, list_nodes_known=[68, 62, 242, 243, 117, 28, 170])

# TGCN Model
model_TGCN = TGCN(adjacency_matrix_PeMS, hidden_dim=32, output_size=len(df_PeMS.columns))
train_loader_TGCN, val_loader_TGCN, test_loader_TGCN, _ = createLoaders(node_0, window_size=_window_size, stride=_stride, prediction_horizon=horizon)
model_path = "./test7_13_without_mask.pkl"

best_model , train_losses, valid_losses = train_model(model_TGCN, train_loader_TGCN, val_loader_TGCN, model_path=model_path, mask=None, num_epochs=num_epochs_TGCN, remove=False)
y_true, y_pred = testmodel(best_model=best_model, test_loader=test_loader_TGCN, meanstd_dict=meanstd_dict, sensor_order_list=node_0.columns)
rmse = 0
for i in range(n_neighbors+1):
    rmse = rmse + np.sqrt(mean_squared_error(y_true[:,:,i], y_pred[:,:,i]))
print(rmse/n_neighbors+1)

model_TGCN = TGCN(adjacency_matrix_PeMS, hidden_dim=32, output_size=len(df_PeMS.columns))
train_loader_TGCN, val_loader_TGCN, test_loader_TGCN, _ = createLoaders(node_1, window_size=_window_size, stride=_stride, prediction_horizon=horizon)
model_path = "./test7_13_mask.pkl"
best_model , train_losses, valid_losses = train_model(model_TGCN, train_loader_TGCN, val_loader_TGCN, model_path=model_path, mask=mask_node_1, num_epochs=num_epochs_TGCN, remove=False)
y_true, y_pred = testmodel(best_model=best_model, test_loader=test_loader_TGCN, meanstd_dict=meanstd_dict, sensor_order_list=node_1.columns)
rmse = 0
for i in range(n_neighbors+1):
    rmse = rmse + np.sqrt(mean_squared_error(y_true[:,:,i], y_pred[:,:,i]))
print(rmse/n_neighbors+1)